In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,row_number,max,min
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

emp_attendance = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "emp_attendance") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

emp_attendance.show()

+--------+----------+-------+
|employee|     dates| status|
+--------+----------+-------+
|      A1|2024-01-01|PRESENT|
|      A1|2024-01-02|PRESENT|
|      A1|2024-01-03|PRESENT|
|      A1|2024-01-04| ABSENT|
|      A1|2024-01-05|PRESENT|
|      A1|2024-01-06|PRESENT|
|      A1|2024-01-07| ABSENT|
|      A1|2024-01-08| ABSENT|
|      A1|2024-01-09| ABSENT|
|      A1|2024-01-10|PRESENT|
|      A2|2024-01-06|PRESENT|
|      A2|2024-01-07|PRESENT|
|      A2|2024-01-08| ABSENT|
|      A2|2024-01-09|PRESENT|
|      A2|2024-01-10| ABSENT|
+--------+----------+-------+



In [7]:
emp_attendance\
    .withColumn("rn1",row_number().over(Window.partitionBy("employee","status").orderBy("employee","dates")))\
    .withColumn("rn2",row_number().over(Window.partitionBy("employee").orderBy("employee","dates")))\
    .groupby(col("employee"),col("status"),(col("rn2")-col("rn1")))\
    .agg(min("dates").alias("from_dt"),max("dates").alias("to_dt"))\
    .select("employee","status","from_dt","to_dt")\
    .show()

+--------+-------+----------+----------+
|employee| status|   from_dt|     to_dt|
+--------+-------+----------+----------+
|      A1| ABSENT|2024-01-04|2024-01-04|
|      A1| ABSENT|2024-01-07|2024-01-09|
|      A1|PRESENT|2024-01-01|2024-01-03|
|      A1|PRESENT|2024-01-05|2024-01-06|
|      A1|PRESENT|2024-01-10|2024-01-10|
|      A2| ABSENT|2024-01-08|2024-01-08|
|      A2| ABSENT|2024-01-10|2024-01-10|
|      A2|PRESENT|2024-01-06|2024-01-07|
|      A2|PRESENT|2024-01-09|2024-01-09|
+--------+-------+----------+----------+

